Adım 1: Veri Setinin Hazırlanması

 Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

 Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek

 Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi

 Adım 5: Weighted Average Recommendation Score'un Hesaplanması
 
 Adım 6: Çalışmanın Fonksiyonlaştırılması


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
def  create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('/content/movie.csv')
    rating = pd.read_csv('/content/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 10000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df


In [ ]:
user_movie_df = create_user_movie_df()

In [ ]:
import pandas as pd
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)

Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

In [ ]:
random_user

61415

In [ ]:
user_movie_df

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),...,Who Framed Roger Rabbit? (1988),Wild Wild West (1999),Willy Wonka & the Chocolate Factory (1971),Witness (1985),"Wizard of Oz, The (1939)","X-Files: Fight the Future, The (1998)",X-Men (2000),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.5,NaN,NaN,4.0,NaN,4.0
2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,5.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,5.0,4.0,4.0,5.0,NaN,NaN,NaN,5.0
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130951.0,NaN,5.0,NaN,NaN,3.5,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NaN olanlar bize hiç izlemediklerini  gösteriyor. Bizim için gerekli olan sadece kullanıcının izleddikleridir.

In [ ]:
random_user_df = user_movie_df[user_movie_df.index == random_user]

In [ ]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

In [ ]:
user_movie_df.loc[user_movie_df.index == random_user,
                  user_movie_df.columns == "Silence of the Lambs, The (1991)"]

title,"Silence of the Lambs, The (1991)"
userId,
61415.0,NaN


In [ ]:
len(movies_watched)

11

Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek

İzlenen filmlere indirgeyelim

In [ ]:
movies_watched_df = user_movie_df[movies_watched]

Kullanıcı ile ortak en az 20 filmi izleyen kullanıcıları yakalayalım.

In [ ]:
user_movie_count = movies_watched_df.T.notnull().sum()

In [ ]:
user_movie_count = user_movie_count.reset_index()

In [ ]:
user_movie_count.columns = ["userId", "movie_count"]

In [ ]:
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False)


,userId,movie_count


In [ ]:
user_movie_count[user_movie_count["movie_count"] == 33].count()


userId         0
movie_count    0
dtype: int64

In [ ]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 20]["userId"]


users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
perc = len(movies_watched) * 60 / 100


Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi
Bunun için 3 adım gerçekleştireceğiz:
1. Sinan ve diğer kullanıcıların verilerini bir araya getireceğiz.
 2. Korelasyon df'ini oluşturacağız.
 3. En benzer bullanıcıları (Top Users) bulacağız

Bütün bir veriyi bir araya getirelim ve korelasyona bakalım

In [ ]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])

In [ ]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()

In [ ]:
corr_df = pd.DataFrame(corr_df, columns=["corr"])

In [ ]:
corr_df.index.names = ['user_id_1', 'user_id_2']


In [ ]:
corr_df = corr_df.reset_index()

In [ ]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)

In [ ]:
top_users = top_users.sort_values(by='corr', ascending=False)

In [ ]:
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

In [ ]:
rating = pd.read_csv('/content/rating.csv')
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')

In [ ]:
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]

Adım 5: Weighted Average Recommendation Score'un Hesaplanması

In [ ]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

In [ ]:
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

,weighted_rating
movieId,


In [ ]:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})


In [ ]:
recommendation_df = recommendation_df.reset_index()

In [ ]:
recommendation_df[recommendation_df["weighted_rating"] > 3.5]

,movieId,weighted_rating


In [ ]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)


In [ ]:
movie = pd.read_csv('/content/movie.csv')
movies_to_be_recommend.merge(movie[["movieId", "title"]])


,weighted_rating,movieId,title


Adım 6: Çalışmanın Fonksiyonlaştırılması

In [ ]:
def  create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('/content/movie.csv')
    rating = pd.read_csv('/content/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 10000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

In [ ]:
user_movie_df = create_user_movie_df()


In [ ]:
def user_based_recommender(random_user, user_movie_df, ratio=60, cor_th=0.65, score=3.5):
    import pandas as pd
    random_user_df = user_movie_df[user_movie_df.index == random_user]
    movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
    movies_watched_df = user_movie_df[movies_watched]
    user_movie_count = movies_watched_df.T.notnull().sum()
    user_movie_count = user_movie_count.reset_index()
    user_movie_count.columns = ["userId", "movie_count"]
    perc = len(movies_watched) * ratio / 100
    users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
